In [1]:
import pandas as pd
from pandas import *
import csv
import nltk
import xlrd
import string
import re
from difflib import SequenceMatcher

In [2]:
### Function 
def is_nan(x):
    return (x is np.nan or x != x)

### DRUG NAMES DICTIONARY FOR US

In [3]:
df_drug_name = pd.read_csv("drug_name_dict.csv", sep=';')
df_drug_name = df_drug_name.drop(columns=['lab_code','trade_name','generic_name',
                                          'lab_code1','lab_code2','lab_code3','lab_code4',
                                         'lab_code5','lab_code6','lab_code7','lab_code8',
                                         'lab_code9','lab_code10','lab_code11','lab_code12','lab_code13'])
df_drug_name.head()

,record_no,trade_name1,trade_name2,trade_name3,trade_name4,trade_name5,trade_name6,trade_name7,trade_name8,trade_name9,...,generic_name1,generic_name2,generic_name3,generic_name4,generic_name5,generic_name6,generic_name7,generic_name8,generic_name9,generic_name10
0,13067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,(+)-calanolide A,calanolide A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31334,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,(+)-enantiomer of DOV 216303,amitifadine (pINN),amitifadine (USAN),amitifadine hydrochloride (USAN),NaN,NaN,NaN,NaN,NaN,NaN
2,34023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,(+)-methadone,(6S)-methadone,(S)-methadone,dextromethadone,d-methadone,NaN,NaN,NaN,NaN,NaN
3,23548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,(+)-thalicarpine,taliblastine,TBL,thaliblastine,thalicarpin,thalicarpine,NaN,NaN,NaN,NaN
4,38423,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,(-)-bambuterol,"bambuterol, (R)",laevo-bambutero,levo-bambuterol,R-bambuterol,NaN,NaN,NaN,NaN,NaN


In [4]:
len(df_drug_name)

7081

In [5]:
df_drug_name.generic_name1[1]

'(+)-enantiomer of DOV 216303'

In [6]:
## CREATE THE LIST OF DRUG NAMES TO EXTRACT
list_df_drug_name =[]
for index, rows in df_drug_name.iterrows():
    for row in rows :
        if not(is_nan(row)) : 
            
            list_df_drug_name.append(tuple((rows['record_no'],row)))

list_drug_name = [i for i in list_df_drug_name if i[0] != i[1]]
list_drug_name[:10]

list_drug_name[:10]

[(13067, '(+)-calanolide A'),
 (13067, 'calanolide A'),
 (31334, '(+)-enantiomer of DOV 216303'),
 (31334, 'amitifadine (pINN)'),
 (31334, 'amitifadine (USAN)'),
 (31334, 'amitifadine hydrochloride (USAN)'),
 (34023, '(+)-methadone'),
 (34023, '(6S)-methadone'),
 (34023, '(S)-methadone'),
 (34023, 'dextromethadone')]

In [7]:
len(list_drug_name)

22444

In [8]:
## CREATE THE DATAFRAME
df_drug_name = pd.DataFrame(list_drug_name, columns=['record_no','Drug_name'])
df_drug_name.head()

,record_no,Drug_name
0,13067,(+)-calanolide A
1,13067,calanolide A
2,31334,(+)-enantiomer of DOV 216303
3,31334,amitifadine (pINN)
4,31334,amitifadine (USAN)


### Clean the Drug Names

In [9]:
# Delete all extra info than drug names
to_delete = ['gene therapy','vaccin','University','School','MAb',',']
count = 0
for x in to_delete:
    for index,row in df_drug_name.iterrows() :
        if x in df_drug_name.Drug_name[index] : 
            count = count+1 
            df_drug_name = df_drug_name.drop(index=index)
            #print(df_drug_name.Drug_name[index])
print(len(df_drug_name))

17050


In [10]:
# Split the drug names where the name is composed, example « flupirtine + opioid »
list_final = []
new_tuple = []
for index,row in df_drug_name.iterrows() :
    rec_num   = row.record_no 
    drug_name = row.Drug_name.split(" + ")
    for name in range(0,len(drug_name)):
        new_tuple.append(tuple((rec_num,drug_name[name].strip())))
list_final.append(new_tuple)   
len(list_final[0])

18219

In [11]:
## CREATE THE CLEAN DATAFRAME
df_drug_name_final = pd.DataFrame(list_final[0], columns=['record_no','drug_name'])
df_drug_name_final.head()

,record_no,drug_name
0,13067,(+)-calanolide A
1,13067,calanolide A
2,31334,(+)-enantiomer of DOV 216303
3,31334,amitifadine (pINN)
4,31334,amitifadine (USAN)


In [12]:
# DELETE DUPLICATES
df_drug_name_final = df_drug_name_final.drop_duplicates(keep=False)
len(df_drug_name_final)

16216

In [13]:
# GSK is for the lab code attached to some drug names
to_delete = ['GSK']
count = 0
for x in to_delete:
    for index,row in df_drug_name_final.iterrows() :
        if x in df_drug_name_final.drug_name[index] : 
            count = count+1 
            df_drug_name_final = df_drug_name_final.drop(index=index)
            #print(df_drug_name.Drug_name[index])
print(len(df_drug_name_final))

16210


In [14]:
#CREATE A NEW EMPTY COLUMN FOR "search_string"
df_drug_name_final['search_string'] = [''] * len(df_drug_name_final)

In [15]:
# CREATE THE SEARCH_STRING VARIABLE
search_str = []
to_delete = ['(R)-(-)-','(S)-(-)-','(6S)-','(-)-','-(R)','(R)-',' (R)-','R-','(S)-',' (S)-',
             ' (pINN)',' (INN)',' (USAN)',
             '18F ','[18F]','F 18','(+)-',]

for index,row in df_drug_name_final.iterrows() :
    for x in to_delete:
        if x in row.drug_name : 
            clean_name = df_drug_name_final.drug_name[index].replace(x,'')
            clean_name = re.sub(r'\([^)]*\)', '', clean_name).strip() #Clean all extra () 
            print(clean_name)
            df_drug_name_final.search_string[index] = clean_name
            #nameD = clean_name.replace('[^a-zA-Z]', '')  

        
df_drug_name_final.head()

calanolide A
enantiomer of DOV 216303
amitifadine
amitifadine
amitifadine hydrochloride
methadone
methadone


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


methadone
thalicarpine
bambuterol
etodolac
etodolic acid
etodolac
etodolac
tenatoprazole
tenatoprazole sodium
tenatoprazole
tenatoprazole sodium
apricitabine
aldoxorubicin
aldoxorubicin
aldoxorubicin hydrochloride
alpha-methylhistamine
-alpha-methylhistamine
-alpha-methylhistamine
7-chloro-benzo[b]thiophene-2-carboxamide
encenicline
encenicline
encenicline hydrochloride
alpha lipoic acid
thioctic acid
nadifloxacin
-nadifloxacin
-nadifloxacin
levonadifloxacin
dianhydrogalactitol
dianhydrogalactitol
duvoglustat
duvoglustat
duvoglustat hydrochloride
indoximod
indoximod
guadecitabine
guadecitabine
guadecitabine sodium
censavudine
censavudine
fenretinide
eniluracil
zuretinol acetate
roledumab
lampalizumab
lampalizumab
donepezil
donepezil hydrochloride
lirilumab
lirilumab
pidilizumab
pidilizumab
napabucasin
alvircept sudotox
abemaciclib
abemaciclib
midafotel
ulodesine
ulodesine
ulodesine succinate
amdoxovir
amdoxovir
sparsentan
sparsentan
stapuldencel-t
enadenotucirev
pinometostat
tesmilifen

diaplasinin
diaplasinin
drospirenone
tiapamil
dimiracetam
dinaciclib
dinaciclib
sodium thiosulfate
sodium thiosulfate
ditekiren
dofequidar
dolcanatide
dolcanatide
doranidazole
doravirine
doravirine
dovitinib
dovitinib lactate
doxycycline
tadalafil
tadalafil
ciclopirox
ciclopirox olamine
travoprost
ketorolac
ketorolac trometamol
salmeterol
naloxone
naloxone hydrochloride
betamethasone
betamethasone
calcitriol
triamcinolone acetonide
triamcinolone acetonide
bimagrumab
bimagrumab
duligotuzumab
duligotuzumab
ecopipam
ecopipam hydrochloride
ecromeximab
edasalonexent
edelfosine
edivoxetine
edivoxetine
edivoxetine hydrochloride
edonerpic
edotecarin
edotecarin
edratide
edratide
eflapegrastim
egaptivon pegol
egaptivon pegol
elacridar
elagolix
elagolix
elagolix sodium
elbasvir
elbasvir
eleclazine
eleclazine
elesclomol
elesclomol
elinafide
elobixibat
elobixibat
elpamotide
elpamotide
elpetrigine
elpetrigine
elsamitrucin
eltoprazine
emapunil
embeconazole
emicizumab
enazadrem
enazadrem phosphate
enc

picoplatin
picoplatin
pictilisib
pictilisib
pilaralisib
pilaralisib
pimasertib
pimasertib
pimasertib
pimasertib
pimodivir
pipendoxifene
pirodavir
piromelatine
crenolanib
crenolanib
crenolanib besylate
plazomicin
plazomicin
plazomicin sulfate
plecanatide
plecanatide
plevitrexed
plinabulin
plinabulin
plovamer
plovamer acetate
pocapavir
peretinoin
pomaglumetad methionil
pomaglumetad methionil
pozanicline
pozanicline
pozanicline tartrate
poziotinib
pracinostat
pratosartan
pregabalin
presatovir
presatovir
pretomanid
pretomanid
prexasertib
prexasertib
progesterone
progesterone
progesterone
progesterone
besipirdine
besipirdine hydrochloride
pumosetrag
itarnafloxin
quarfloxin
quinupristin
quisinostat
quisinostat
quisinostat hydrochloride
quizartinib
quizartinib
quizartinib dihydrochloride
rabacfosadine
rabacfosadine succinate
rabeximod
racotumomab
radalbuvir
radezolid
radezolid
radezolid hydrochloride
fadolmidine
fadolmidine hydrochloride
radretumab
priralfinamide
ralimetinib
ralimetinib
ralim

...

linezolid
florbetaben
florbetaben F18
desmethylsibutramine
sibutramine
estradiol
estradiol
recoflavone
emixustat
emixustat
emixustat hydrochloride
nifedipine
clotrimazole
lorcaserin
lorcaserin hydrochloride
fosphenytoin
fosphenytoin sodium
remtolumab
melphalan
melphalan
brincidofovir
brincidofovir
brimapitide
carotuximab
ocaratuzumab
ocaratuzumab
foralumab
namilumab
carlumab
carlumab
tralokinumab
brodalumab
brodalumab
sarilumab
sarilumab
opicinumab
opicinumab
monalizumab
vatelizumab
tesidolumab
burosumab
teprotumumab
landogrozumab
landogrozumab
bavituximab
bavituximab
ropeginterferon alfa-2b
pegsunercept
ozanimod
ozanimod
ozanimod hydrochloride
onercept
onercept
abicipar pegol
acalabrutinib
acamprosate
acamprosate calcium
acebilustat
adalimumab
adalimumab
adalimumab
adalimumab
adalimumab
adalimumab
adalimumab
adalimumab
adalimumab
adalimumab
lucinactant
alprazolam
alprazolam
alprostadil
amolimogene bepiplasmid
anagrelide
anagrelide hydrochloride
vobarilizumab
vobarilizumab
volociximab


senrebotase
senrebotase
mirogabalin
mirogabalin
mirogabalin besylate
naldemedine
naldemedine
naldemedine tosylate
nelociguat
neramexane
neramexane mesylate
lanoteplase
odanacatib
odanacatib
olesoxime
olesoxime
omipalisib
omipalisib
orazipone
ornithine phenylacetate
osilodrostat
osilodrostat
osilodrostat phosphate
oxybutynin
ozoralizumab
ozoralizumab
parogrelil
pelitinib
pibrozelesin
pibrozelesin hydrobromide
piclamilast
piritrexim
ponesimod
ponesimod
pradigastat
pradigastat
pradigastat sodium
prednisolone
radafaxine
radafaxine hydrochloride
ranirestat
remetinostat
remetinostat
reparixin
reparixin
revamilast
rilapladib
rilapladib
risarestat
rituximab
rituximab
rituximab
rituximab
rituximab
rituximab
rituximab
rituximab
rituximab
rituximab
rituximab
rituximab
romazarit
romazarit
ronacaleret
ronacaleret hydrochloride
ropivacaine
seliciclib
rozrolimupab
sarecycline
sarecycline
sarecycline hydrochloride
semapimod
setileuton
setileuton
sifalimumab
sifalimumab
solabegron
solabegron hydrochlor

fentanyl
fentanyl citrate
flupirtine
flupirtine maleate
bupivacaine
bupivacaine
sivelestat
sivelestat
sivelestat sodium
leuprorelin
leuprolide acetate
estradiol
estradiol
ulipristal acetate
ulipristal acetate
metoclopramide
selegiline
selegiline
selegiline hydrochloride
zotarolimus
zotarolimus
progesterone
progesterone
tacrolimus
enalapril
oxcarbazepine
oxcarbazepine
carbamazepine
ketamine
vismodegib
vismodegib
estradiol
estradiol
nilvadipine
fenspiride
estradiol
estradiol
estradiol
estradiol
estradiol
estradiol
estradiol
estradiol
estradiol
estradiol
estradiol
estradiol
estradiol
estradiol
estradiol
estradiol
doxorubicin
doxorubicin
fibrinogen
squalamine
squalamine lactate
melphalan
melphalan
naloxone
naloxone hydrochloride
melagatran
ximelagatran
ximelagatran
ofloxacin
amantadine
amantadine hydrochloride
bupivacaine
bupivacaine
ketoconazole
levoketoconazole
econazole
estradiol
estradiol
fenticonazole
fenticonazole nitrate
fulvestrant
estradiol
estradiol
estradiol
estradiol
fenofibrat

nintedanib
nintedanib
nintedanib esylate
rolapitant
rolapitant
rolapitant hydrochloride
nitroglycerin
phentolamine
phentolamine mesylate
vasopressin
epoprostenol
epoprostenol sodium
clozapine
imiquimod
imiquimod
miglustat
miglustat
azacitidine
vatiquinone
vatiquinone
nelfinavir
ganciclovir
hyaluronidase
hyaluronidase
fomivirsen
naltrexone
meloxicam
salbutamol
salbutamol
estradiol
estradiol
calcitonin
calcitonin
teriparatide
astodrimer
astodrimer
astodrimer sodium
corticorelin
corticorelin acetate
omalizumab
omalizumab
levosalbutamol
levalbuterol
levalbuterol tartrate
albuterol
-albuterol
salbutamol
-salbutamol
oxycodone
oxycodone
finafloxacin
zabofloxacin
thymalfasin
tipifarnib
sumatriptan
daclizumab
ondansetron
acitazanolast
ferric citrate
ibritumomab tiuxetan
triamcinolone acetonide
triamcinolone acetonide
paclitaxel
diclofenac
diclofenac
diclofenac potassium
diclofenac sodium
dithranol
anthralin
ondansetron
hydrocodone
goserelin
zolpidem
zolpidem tartrate
diclofenac
diclofenac
diclo

,record_no,drug_name,search_string
0,13067,(+)-calanolide A,calanolide A
1,13067,calanolide A,
2,31334,(+)-enantiomer of DOV 216303,enantiomer of DOV 216303
3,31334,amitifadine (pINN),amitifadine
4,31334,amitifadine (USAN),amitifadine


In [16]:
len(df_drug_name_final)

16210

In [17]:
#Save the rest of the drug names, if they does not need any cleaning
for index,row in df_drug_name_final.iterrows() :           
    if (df_drug_name_final.search_string[index] == ''):
        clean_name = re.sub(r'\([^)]*\)', '', df_drug_name_final.drug_name[index]).strip()
        df_drug_name_final.search_string[index] = clean_name
        
df_drug_name_final.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,record_no,drug_name,search_string
0,13067,(+)-calanolide A,calanolide A
1,13067,calanolide A,calanolide A
2,31334,(+)-enantiomer of DOV 216303,enantiomer of DOV 216303
3,31334,amitifadine (pINN),amitifadine
4,31334,amitifadine (USAN),amitifadine


In [18]:
df_drug_name_final.to_excel('drug_names_final.xlsx')

## TEST SIMILARITY

In [ ]:
def csv_from_excel():
    wb = xlrd.open_workbook('intervention_arm_sample.xlsx')
    sh = wb.sheet_by_name('Sheet1')
    your_csv_file = open('intervention_arm_sample.csv', 'w')
    wr = csv.writer(your_csv_file, quoting=csv.QUOTE_ALL)

    for rownum in range(sh.nrows):
        wr.writerow(sh.row_values(rownum))

    your_csv_file.close()

# runs the csv_from_excel function:
csv_from_excel()

In [ ]:
### calculate similarity with "SequenceMatcher" where space isJunk 
def similar(a, b):
    return SequenceMatcher(lambda x: x == " ", a.lower(), b.lower()).ratio()

In [ ]:
intervention_arm_sample = pd.read_csv("intervention_arm_sample.csv", sep=',')
intervention_arm_sample = intervention_arm_sample.drop(columns=['Unnamed: 0'])
intervention_arm_sample.head()

In [ ]:
df_drug_name_final

arm_label
intervention_name


In [ ]:
#len(intervention_arm_sample)

for index in range(0,10):
    arm_label_desc = arm_label[index]
    intervention_name_desc = intervention_name[index]
    
    sim = similar(arm_label_desc,intervention_name_desc)

In [ ]:
intervention_arm_sample.to_excel('intervention_arm_sample.xlsx')